In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

In [ ]:
df1=pd.read_excel('tweets_beforecoup.xlsx', engine='openpyxl')

In [ ]:
df1=df1[['content','date','polarity']]

In [ ]:
df1=df1.dropna()

In [ ]:
df1.copy()
df1["polarity"].replace({"nt": "Neutral", "p": "Positive","n":"Negative",'np':'Negative'}, inplace=True)

In [ ]:
plt.pie(df1['polarity'].value_counts(),labels=['Neutral','Positive','Negative'],autopct='%1.1f%%')
plt.show()
df1['polarity'].value_counts()


In [ ]:
df2=pd.read_excel('tweets_aftercoup.xlsx', engine='openpyxl')

In [ ]:
df2

In [ ]:
df2=df2[['content','date','polarity ']]

In [ ]:
df2=df2.rename(columns = {'polarity ':'polarity'})

In [ ]:
df2.copy()
df2["polarity"].replace({"nt": "Neutral", "p": "Positive","n":"Negative",'np':'Negative'}, inplace=True)

In [ ]:
plt.pie(df2['polarity'].value_counts(),labels=['Neutral','Positive','Negative'],autopct='%1.1f%%')
plt.show()
df2['polarity'].value_counts()

In [ ]:
df=pd.concat([df1,df2])

In [ ]:
df=df.reset_index()

In [ ]:
df

# PREPROCESSING

In [ ]:
import re

In [ ]:
df['content'] = [token.lower() for token in df['content']]
df.head(5)

In [ ]:
found = df[df['content'].str.contains('\n')]
found.count()

In [ ]:
df['content'] = df['content'].replace('\n', '', regex=True)
found = df[df['content'].str.contains('\n')]
found.count()

In [ ]:
found = df[df['content'].str.contains('@')]
found.count()

In [ ]:
df['content'] = df['content'].replace('@[A-Za-z0-9_]+', '', regex=True).replace('@[A-Za-z0-9_]', '', regex=True)
found = df[df['content'].str.contains('@')]
found.count()

In [ ]:
df[df['content'].str.contains('@')]

In [ ]:
df['content'] = df['content'].replace('@', '', regex=True)
found = df[df['content'].str.contains('@')]
found.count()

In [ ]:
df['content'] = df['content'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
found = df[df['content'].str.contains('http')]
found.count()

# Remove Punctuations, Emojis and Numbers

In [ ]:
sentences = df['content'].copy()
new_sent = []
i = 0
for sentence in sentences:
    new_sentence = re.sub('[0-9]+', '', sentence)
    new_sent.append(new_sentence)
    i += 1
    
df['content'] = new_sent
df

In [ ]:
import string

table = str.maketrans('', '', string.punctuation)
sentences = df['content'].copy()
new_sent = []
for sentence in sentences:
    words = sentence.split()
    stripped = [w.translate(table) for w in words]
    new_sent.append(stripped)


In [ ]:
df['content'] = new_sent
df

# Tokenization

In [ ]:
import time
import logging

from zemberek import (
    TurkishSpellChecker,
    TurkishSentenceNormalizer,
    TurkishSentenceExtractor,
    TurkishMorphology,
    TurkishTokenizer
)

logger = logging.getLogger(__name__)

morphology = TurkishMorphology.create_with_defaults()
normalizer = TurkishSentenceNormalizer(morphology)
extractor = TurkishSentenceExtractor()

# Normalization

In [ ]:
def normalize_long_text(text):
    normalized_sentences = [normalizer.normalize(word) for word in text]
    normalized_text = " ".join(normalized_sentences)
    return normalized_text

In [ ]:
sentences = df['content'].copy()
new_sent = []
start = time.time()

for token in sentences:   
    if token.count('') > 0:
        token = list(filter(('').__ne__, token))
    new_token = normalize_long_text(token)
    new_sent.append(new_token)

logger.info(f"Sentences normalized in: {time.time() - start} s")

# Stopwords

In [ ]:
from nltk.corpus import stopwords


stops = set(stopwords.words('turkish'))
print(stops)

In [ ]:
splitted_words = []
for sent in new_sent:
    words = sent.split()
    splitted_words.append(words)

In [ ]:
clean_sent = []
for sentence in splitted_words:
    new_sentence = [w for w in sentence if w not in stops]
    clean_sent.append(new_sentence)

# Lemmatization

In [ ]:
for token in clean_sent:
    j = 0
    for word in token:
        new_word = word.replace('"', '').replace("’", '').replace("'", '').replace("”", '')
        token[j] = new_word
        j += 1

In [ ]:
import zeyrek


analyzer = zeyrek.MorphAnalyzer()
lem_sent = []
for sent in clean_sent:
    normalized_sent = []
    for word in sent:
        if word == '':
            continue
        else:
            lem_word = analyzer.lemmatize(word)
            normalized_sent.append(lem_word[0][1][0])
    lem_sent.append(normalized_sent)

In [ ]:
x = lem_sent.copy()
for sent in x:
    i = 0
    for token in sent:
        sent[i] = token.lower()
        i += 1
lem_sent = x

In [ ]:
lem_sent = list(filter(('').__ne__, lem_sent))

In [ ]:
df['content'] = lem_sent
df['content'].head(5)

In [ ]:
df['content'] = df.content.apply(' '.join)

In [ ]:
df.to_csv('tweets_lemmatized.csv')